In [7]:
# %pip install tensorflow opencv-python matplotlib pyautogui numpy
print("api is working fine")

In [8]:
import tensorflow as tf
import numpy as np
import pyautogui
import cv2

# Load Model

In [9]:
interpreter = tf.lite.Interpreter(model_path='./model/3.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [10]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [11]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [12]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [13]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [14]:
cap = cv2.VideoCapture(0)
body_parts = {5: 'left_shoulder', 7: 'left_elbow', 9: 'left_wrist', 11: 'left_hip', 6: 'right_shoulder', 8: 'right_elbow', 10: 'right_wrist', 12: 'right_hip', 13: 'left_knee', 15: 'left_ankle', 14: 'right_knee', 16: 'right_ankle'}

frame_count = 0
SKIP_FRAMES = 2

while cap.isOpened():
    ret, frame = cap.read()

    frame_count += 1
    if frame_count % SKIP_FRAMES != 0:
        continue
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    keypoints = {body_parts[i]: keypoints_with_scores[0][0][i][:2] for i in body_parts}

    # Calculate angles
    left_hand_straight_angle = calculate_angle(keypoints['left_shoulder'], keypoints['left_elbow'], keypoints['left_wrist'])
    left_hand_up_angle = calculate_angle(keypoints['left_hip'], keypoints['left_shoulder'], keypoints['left_elbow'])
    left_hand_up_straight = calculate_angle(keypoints['left_hip'], keypoints['left_shoulder'], keypoints['left_wrist'])
    right_hand_straight_angle = calculate_angle(keypoints['right_shoulder'], keypoints['right_elbow'], keypoints['right_wrist'])
    right_hand_up_angle = calculate_angle(keypoints['right_hip'], keypoints['right_shoulder'], keypoints['right_elbow'])
    right_hand_up_straight = calculate_angle(keypoints['right_hip'], keypoints['right_shoulder'], keypoints['right_wrist'])
    left_leg_angle = calculate_angle(keypoints['left_hip'], keypoints['left_knee'], keypoints['left_ankle'])
    right_leg_angle = calculate_angle(keypoints['right_hip'], keypoints['right_knee'], keypoints['right_ankle'])

    left_hand_straight = left_hand_straight_angle > 160
    left_hand_up = 60 < left_hand_up_angle < 130
    right_hand_straight = right_hand_straight_angle > 160
    right_hand_up = 60 < right_hand_up_angle < 130
    is_sitting = left_leg_angle < 100 and right_leg_angle < 100


    # key mapping based on poses
    if(left_hand_up and left_hand_straight):
        cv2.putText(frame, 'Left hand up', 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        pyautogui.press('left')

    if(right_hand_up and right_hand_straight):
        cv2.putText(frame, 'Right hand up', 
                    (10,120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        pyautogui.press('right')

    if(is_sitting):
        cv2.putText(frame, 'Sitting', 
                    (10,180), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        pyautogui.press('down')

    if(left_hand_up_straight > 160 and right_hand_up_straight > 160):
        cv2.putText(frame, 'Both hands up', 
                    (10,240), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        pyautogui.press('up')

    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(4) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/aarib/Desktop/My_Files/Programming/Hackathon/Vercera_2024/.venv/lib/python3.10/site-packages/cv2/qt/plugins"
